In [ ]:
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import commonteamroster,shotchartdetail,boxscoreadvancedv2,boxscoreplayertrackv2,playergamelog,boxscorematchups,leaguegamefinder,playbyplayv2
import json
import pandas as pd
from pandas.io.json import json_normalize

custom_headers = {
        'Host': 'stats.nba.com',
        'Connection': 'keep-alive',
        'Accept': 'application/json, text/plain, */*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'Referer': 'https://stats.nba.com/',
        "x-nba-stats-origin": "stats",
        "x-nba-stats-token": "true",
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        }

team_ids = json_normalize(teams.get_teams())['id'].unique().tolist()
df = pd.DataFrame()

def getroster(i):
    roster = pd.DataFrame(json.loads(commonteamroster.CommonTeamRoster(team_id=i,season='2019-20',headers=custom_headers,timeout=1000).get_json())['resultSets'][0]['rowSet'],
                                   columns=json.loads(commonteamroster.CommonTeamRoster(team_id=i,season='2019-20',headers=custom_headers,
                                                                                timeout=1000).get_json())['resultSets'][0]['headers'])
    return roster

for i in team_ids:
    result = getroster(i)
    df = pd.concat([df,result])

df

# getroster(1610612749)


In [ ]:
######### Gets roster player ids for all teams #############
players_full_list = pd.DataFrame()

team_ids = json_normalize(teams.get_teams())['id'].unique().tolist()
team_names = json_normalize(teams.get_teams())[['id','full_name','abbreviation']]

for i in team_ids:
    roster_response = pd.DataFrame(json.loads(commonteamroster.CommonTeamRoster(team_id=i,
                                                                                season='2019-20',
                                                                                headers=custom_headers,
                                                                                timeout=1800).get_json())['resultSets'][0]['rowSet'],
                                   columns=json.loads(commonteamroster.CommonTeamRoster(team_id=i,
                                                                                season='2019-20',
                                                                                headers=custom_headers,
                                                                                timeout=1800).get_json())['resultSets'][0]['headers'])
#     data = json.loads(roster_response)['resultSets'][0]['rowSet']
#     cols = json.loads(roster_response)['resultSets'][0]['headers']
#     roster_df = pd.DataFrame.from_records(data)
#     roster_df.columns = cols
    players_full_list = pd.concat([players_full_list,roster_response])
players_full_list.head(10)

In [ ]:
############### Gets all shot attempts/locations from team roster id list ##############
joined_rosters = pd.merge(players_full_list,team_names,how='inner',left_on='TeamID',right_on='id')
players = joined_rosters.groupby(['PLAYER_ID','id']).apply(list).reset_index(name='new')
player_ids = players['PLAYER_ID'].tolist()
team_ids = players['id'].tolist()

shot_full_list = pd.DataFrame()

for i,t in zip(player_ids,team_ids):
    try:
        shot_response = pd.DataFrame(json.loads(shotchartdetail.ShotChartDetail(player_id=i,context_measure_simple='FGA',
                                                        season_nullable='2019-20',
                                                        team_id=t,headers=custom_headers,timeout=20).get_json())['resultSets'][0]['rowSet'],
                                     columns=json.loads(shotchartdetail.ShotChartDetail(player_id=i,context_measure_simple='FGA',
                                                        season_nullable='2019-20',
                                                        team_id=t,headers=custom_headers,timeout=20).get_json())['resultSets'][0]['headers'])
#         shot_data = json.loads(shot_response)['resultSets'][0]['rowSet']
#         shot_cols = json.loads(shot_response)['resultSets'][0]['headers']
#         shot_df = pd.DataFrame.from_records(shot_data)
#         shot_df.columns = shot_cols
    except ValueError:
        continue
    else: 
        shot_full_list = pd.concat([shot_full_list,shot_response])
shot_full_list = pd.merge(shot_full_list,team_names,how='left',left_on='TEAM_ID',right_on='id')
shot_full_list.head(50)
# shot_full_list.to_csv('shotchartdetail.csv')

In [ ]:
game_ids = pd.DataFrame(json.loads(leaguegamefinder.LeagueGameFinder(season_nullable='2019-20',league_id_nullable='00',
                                                          season_type_nullable='Regular Season',headers=custom_headers,
                                                                     timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(leaguegamefinder.LeagueGameFinder(season_nullable='2019-20',league_id_nullable='00',
                                                                 season_type_nullable='Regular Season',headers=custom_headers,
                                                                 timeout=1800).get_json())['resultSets'][0]['headers'])['GAME_ID'].unique().tolist()

touchpoints_list = pd.DataFrame()

# ########## Player touch points ######################

for g in game_ids:
    try:
        touch_response = pd.DataFrame(json.loads(boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['headers'])
    except ValueError:
        continue
    else: 
        touchpoints_list = pd.concat([touchpoints_list,touch_response])

touchpoints_list['new_min'] = touchpoints_list['MIN'].str.split(':',n=1,expand=True)[0]
touchpoints_list['new_sec'] = touchpoints_list['MIN'].str.split(':',n=1,expand=True)[1]
# touchpoints_list.to_csv('touchesdetail.csv')



In [ ]:
########## Player game by game stats #################
play_by_play_list = pd.DataFrame()

for p in player_ids:
    play_repsonse = pd.DataFrame(json.loads(playergamelog.PlayerGameLog(player_id=p,season='2019-20',headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(playergamelog.PlayerGameLog(player_id=p,season='2019-20',headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['headers'])
    except ValueError:
        continue
    else:
        play_by_play_list = pd.concat([play_by_play_list,play_repsonse])

play_by_play_list.head(10)

In [ ]:
######### Player game matchup players ################
# pd.DataFrame(json.loads(boxscorematchups.BoxScoreMatchups(game_id='0021900971',headers=custom_headers,timeout=20).get_json())['resultSets'][0]['rowSet'],
#             columns=json.loads(boxscorematchups.BoxScoreMatchups(game_id='0021900971',headers=custom_headers,timeout=20).get_json())['resultSets'][0]['headers'])


In [ ]:
########## Play by play #################
# pd.DataFrame(json.loads(playbyplayv2.PlayByPlayV2(game_id='0021900971',headers=custom_headers,timeout=20).get_json())['resultSets'][0]['rowSet'],
#             columns=json.loads(playbyplayv2.PlayByPlayV2(game_id='0021900971',headers=custom_headers,timeout=20).get_json())['resultSets'][0]['headers'])
